In [49]:
import os

import numpy as np
import pandas as pd

In [67]:
BACKTEST_CCY = 'USDRUB'
BACKTEST_DAYS = 5

PATH_TEXTS = 'data/telegram'
PATH_SPOT_PNL = 'data/moex'
PATH_OPT_PNL = 'data/pnl'

In [51]:
# Get all available data_sources
sources = os.listdir(PATH_TEXTS)
sources

['cbonds.csv',
 'themovchans.csv',
 'headlines_QUANTS.csv',
 'War_Wealth_Wisdom.csv',
 'mmi.csv',
 'vts.csv',
 'signal.csv',
 '.gitignore',
 'rshb_invest.csv',
 'Alfa_Wealth.csv',
 'sky_bond.csv',
 'bitkogan.csv']

In [52]:
# Generate dataframes
data_sources = []

for s in sources:
    if s != '.gitignore':
        source_data = pd.read_csv(f'{PATH_TEXTS}/{s}')
        data_sources.append((s.split('.')[0], source_data))

In [128]:
spot_pnl = pd.read_csv(f'{PATH_SPOT_PNL}/{BACKTEST_CCY}.csv')
spot_pnl.dropna(inplace=True)
spot_pnl['date_start'] = pd.to_datetime(spot_pnl['timestamp']).dt.strftime('%Y-%m-%d')
spot_pnl.drop(['timestamp'], axis=1, inplace=True)

spot_pnl.drop_duplicates(subset=['date_start'], keep='first', inplace=True)
spot_pnl.reset_index(inplace=True, drop=True)

spot_pnl['pnl'] = spot_pnl['Bid'].div(spot_pnl['Bid'].shift(1)) - 1
spot_pnl['pnl_sign'] = spot_pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
spot_pnl

,Bid,Ask,date_start,pnl,pnl_sign
0,94.994073,96.879993,2022-03-01,NaN,0
1,95.803033,95.971040,2022-03-02,0.008516,1
2,114.920925,114.999880,2022-03-03,0.199554,1
3,105.000043,108.100490,2022-03-04,-0.086328,0
4,116.958480,117.096750,2022-03-09,0.113890,1
...,...,...,...,...,...
153,61.484698,61.978255,2022-10-18,-0.003440,0
154,61.588507,62.014587,2022-10-19,0.001688,1
155,61.600813,62.025917,2022-10-20,0.000200,1
156,61.144888,61.667988,2022-10-21,-0.007401,0


In [120]:
# Create target variable dataframe
pnl = pd.read_csv(f'{PATH_OPT_PNL}/Backtest_{BACKTEST_CCY}_{BACKTEST_DAYS}_days.txt')
pnl['date_start'] = pd.to_datetime(pnl['date_start']).dt.strftime('%Y-%m-%d')
pnl['pnl_sign'] = pnl['pnl'].apply(lambda x: 1 if x >= 0 else 0)
pnl

,date_start,pnl,pnl_sign
0,2021-01-04,291264.097914,1
1,2021-01-05,-411993.830320,0
2,2021-01-06,521491.686795,1
3,2021-01-08,62842.634116,1
4,2021-01-11,-537598.706217,0
...,...,...,...
245,2021-12-20,-407527.554561,0
246,2021-12-21,-176881.417077,0
247,2021-12-22,-206943.414418,0
248,2021-12-23,23073.596468,1


In [121]:
# Get balance of the sample
# pnl['pnl_sign'].sum() / pnl.shape[0]
spot_pnl['pnl_sign'].sum() / spot_pnl.shape[0]

0.4741847826086957

In [12]:
# df = df[(df['date'] >= '2022-03-01') & (df['date'] <= '2022-10-31')]

In [125]:
data_sources[0][1]

,id,date,text
0,9,2017-07-28T10:56:14,Cbonds.ru запустило канал в Telegram. Планиру...
1,10,2017-08-03T15:28:17,Облигационный бюллетень Cbonds – все данные о ...
2,11,2017-08-03T15:41:02,Cbonds prepared monthly report: CBONDS GLOBAL ...
3,12,2017-08-03T18:11:32,Дайджест Cbonds от 3 августа:«Группа Компаний...
4,13,2017-08-04T12:42:41,Совкомбанк открыл книгу заявок на вторичное ра...
...,...,...,...
11997,13124,2022-11-04T12:38:56,​​ #РейтингиСНГ 🗂 Присвоение / Изменение рейт...
11998,13125,2022-11-04T13:17:23,#CbondsNewsletter 🌎📰 В октябре объем новых вы...
11999,13126,2022-11-04T14:30:00,"​​ 🏢Онлайн-семинар «Ключевые факты, финансовые..."
12000,13128,2022-11-05T11:30:11,#Аналитика 🏆 Research Hub Cbonds: самое попу...


In [14]:
# df = pnl
# for source_name, source_dataframe in data_sources:
#     df[f'{source_name}'] = None
    
#     i = 0
#     texts = []
#     for _, row in source_dataframe.iterrows():
#         if i >= df.shape[0]:
#             break

#         if pd.to_datetime(row['date']).strftime('%Y-%m-%d') <= df['date_start'][i]:
#             texts.append(row['text'])
#         else:
#             df.at[i, source_name] = texts
#             texts = []
#             i += 1
#             continue

In [129]:
df = spot_pnl
for source_name, source_dataframe in data_sources:
    df[f'{source_name}'] = ''
    
    i = 0
    texts = ''
    for _, row in source_dataframe.iterrows():
        if i >= df.shape[0]:
            break

        if pd.to_datetime(row['date']).strftime('%Y-%m-%d') <= df['date_start'][i]:
            texts += ' ' + row['text']
        else:
            df.at[i, source_name] = texts
            texts = ''
            i += 1
            continue

In [130]:
df = df.iloc[1:]
df

,Bid,Ask,date_start,pnl,pnl_sign,cbonds,themovchans,headlines_QUANTS,War_Wealth_Wisdom,mmi,vts,signal,rshb_invest,Alfa_Wealth,sky_bond,bitkogan
1,95.803033,95.971040,2022-03-02,0.008516,1,#НовостиКомпаний ⚡️ Российские компании: осн...,,,,"🟢🔴 - Заокеанские индексы закрылись в минусе, ...",,#Макро \n📊 Рынок смартфонов в РФ по производи...,НОВОСТИ ДНЯ 📊 США и страны МЭА высвободят 60 ...,,​​ PetroRio\n \nКрупнейшая бразильская незави...,Друзья! События в мире разворачиваются с тако...
2,114.920925,114.999880,2022-03-03,0.199554,1,⚡️Важное на рынках: 🏛Банк России до конца год...,,,,‼️ ( Reuters ) Российский Центробанк ввел с 3...,​​ Борьба на валютном рынке . \nГрафик рубль/...,#GAZP #ROSN #Санкции \n🇺🇸 Республиканцы ...,ГЛАВНОЕ В РОССИИ 📈 Правительство приостановит...,,"​​ Alibaba\n \n Alibaba Group , наверно, сама...","Fitch. Что за диверсия? В Fitch опасаются ,..."
3,105.000043,108.100490,2022-03-04,-0.086328,0,"⚡️Важное на рынках: 🏛💵Банк России, дополнител...",,,,‼️- Банк России принял решение не возобновля...,,"$=109,54р. RTS 0,00% BRENT +3,35%📌 LIVE Карт...",НОВОСТИ ДНЯ 📊 ФРС ожидает роста инфляции в СШ...,,"​​ Не так давно я рассказывал о том, что об...",Как все вышеперечисленное влияет на жизнь кон...
4,116.958480,117.096750,2022-03-09,0.113890,1,#РейтингиМир 📊 Рейтинговые действия в иностра...,,,Ещё более драматичная ситуация с мировыми цен...,‼️ Все карты международных платежных систем V...,Недорубли\n Две новости. Visa и Mastercard ро...,#СМИ \n📰 Американские СМИ Bloomberg и CNN объ...,ГЛАВНОЕ В РОССИИ 📈 Торги на основной сессии М...,,​​ На счет статьи РБК об ограничениях и сло...,Журналисты Bloomberg прощаются с Россией Аген...
5,120.600105,120.751750,2022-03-10,0.031136,1,💻 JPMorgan Chase & Co. отказался от роли кон...,,,,🟢 - Нефть в последние сутки крайне волатильн...,,#СМИ \n📰 Reuters узнал о планах ЕС отказатьс...,ГЛАВНОЕ В РОССИИ 📊 Инфляция в России за недел...,Рынок ждёт открытия торгов на следующей недел...,,"Годовая инфляция в России достигла 10,4% . Ро..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,61.484698,61.978255,2022-10-18,-0.003440,0,⏰ Last-call: НЕДЕЛЯ до XI конференции «Ипоте...,,,,РЫНОК ЖИЛОЙ НЕДВИЖИМОСТИ В МОСКВЕ: ДИНАМИКА В...,,🔦 Фокус недели #ФН Сегодня\n❗️ XX съезд Комп...,📈 Металлурги против роста тарифов ОАО «РЖД»....,В Дубай через Альфа-Капитал ( продолжение )В...,,График индекса DXY @bitkogan Доходность 10-...
154,61.588507,62.014587,2022-10-19,0.001688,1,#Еврооблигации #ОФЗ 📝 Выплаты купонов по ОФ...,,,,ИНФЛЯЦИЯ В UK: ЦЕНОВОЙ КОШМАР ВНОВЬ УСИЛИВАЕТ...,,"$=61,87р. RTS -2,61% BRENT +0,97%📌 LIVE Карт...",📈 «Ведомости»: «Роснано» могут расформироват...,,,На рынках продолжается фиеста Великобритании...
155,61.600813,62.025917,2022-10-20,0.000200,1,#Еврооблигации #ОФЗ 📝 Выплаты купонов по ОФ...,,,Акции Tesla упали после публикации отчетности...,🔴🟢 - Снижение в пределах процента на WallStre...,,🔦 Фокус недели #ФН Сегодня\n🇷🇺 Россия: засед...,📈 Росстат сообщил о росте цен промпроизводит...,,,А теперь о рынках. Что интересного там?▪️Инде...
156,61.144888,61.667988,2022-10-21,-0.007401,0,​​ #РейтингиСНГ 🗂 Присвоение / Изменение рей...,,,,🔴🟢 - Доходности основных суверенных бенчмарко...,,🔦 Фокус недели #ФН Сегодня\n🇷🇺 Россия: засед...,📈 Softline окончательно разделилась на две к...,,,Еще немного о рынках. Всемирно известный эко...


In [131]:
df = df[~df.isin([''])].dropna(axis=1, how='all')

In [132]:
# df_final = df.drop(['cbonds', 'signal', 'rshb_invest', 'Alfa_Wealth', 'bitkogan'], axis=1)
df_final = df
df_final

,Bid,Ask,date_start,pnl,pnl_sign,cbonds,War_Wealth_Wisdom,mmi,vts,signal,rshb_invest,Alfa_Wealth,sky_bond,bitkogan
1,95.803033,95.971040,2022-03-02,0.008516,1,#НовостиКомпаний ⚡️ Российские компании: осн...,NaN,"🟢🔴 - Заокеанские индексы закрылись в минусе, ...",NaN,#Макро \n📊 Рынок смартфонов в РФ по производи...,НОВОСТИ ДНЯ 📊 США и страны МЭА высвободят 60 ...,NaN,​​ PetroRio\n \nКрупнейшая бразильская незави...,Друзья! События в мире разворачиваются с тако...
2,114.920925,114.999880,2022-03-03,0.199554,1,⚡️Важное на рынках: 🏛Банк России до конца год...,NaN,‼️ ( Reuters ) Российский Центробанк ввел с 3...,​​ Борьба на валютном рынке . \nГрафик рубль/...,#GAZP #ROSN #Санкции \n🇺🇸 Республиканцы ...,ГЛАВНОЕ В РОССИИ 📈 Правительство приостановит...,NaN,"​​ Alibaba\n \n Alibaba Group , наверно, сама...","Fitch. Что за диверсия? В Fitch опасаются ,..."
3,105.000043,108.100490,2022-03-04,-0.086328,0,"⚡️Важное на рынках: 🏛💵Банк России, дополнител...",NaN,‼️- Банк России принял решение не возобновля...,NaN,"$=109,54р. RTS 0,00% BRENT +3,35%📌 LIVE Карт...",НОВОСТИ ДНЯ 📊 ФРС ожидает роста инфляции в СШ...,NaN,"​​ Не так давно я рассказывал о том, что об...",Как все вышеперечисленное влияет на жизнь кон...
4,116.958480,117.096750,2022-03-09,0.113890,1,#РейтингиМир 📊 Рейтинговые действия в иностра...,Ещё более драматичная ситуация с мировыми цен...,‼️ Все карты международных платежных систем V...,Недорубли\n Две новости. Visa и Mastercard ро...,#СМИ \n📰 Американские СМИ Bloomberg и CNN объ...,ГЛАВНОЕ В РОССИИ 📈 Торги на основной сессии М...,NaN,​​ На счет статьи РБК об ограничениях и сло...,Журналисты Bloomberg прощаются с Россией Аген...
5,120.600105,120.751750,2022-03-10,0.031136,1,💻 JPMorgan Chase & Co. отказался от роли кон...,NaN,🟢 - Нефть в последние сутки крайне волатильн...,NaN,#СМИ \n📰 Reuters узнал о планах ЕС отказатьс...,ГЛАВНОЕ В РОССИИ 📊 Инфляция в России за недел...,Рынок ждёт открытия торгов на следующей недел...,NaN,"Годовая инфляция в России достигла 10,4% . Ро..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,61.484698,61.978255,2022-10-18,-0.003440,0,⏰ Last-call: НЕДЕЛЯ до XI конференции «Ипоте...,NaN,РЫНОК ЖИЛОЙ НЕДВИЖИМОСТИ В МОСКВЕ: ДИНАМИКА В...,NaN,🔦 Фокус недели #ФН Сегодня\n❗️ XX съезд Комп...,📈 Металлурги против роста тарифов ОАО «РЖД»....,В Дубай через Альфа-Капитал ( продолжение )В...,NaN,График индекса DXY @bitkogan Доходность 10-...
154,61.588507,62.014587,2022-10-19,0.001688,1,#Еврооблигации #ОФЗ 📝 Выплаты купонов по ОФ...,NaN,ИНФЛЯЦИЯ В UK: ЦЕНОВОЙ КОШМАР ВНОВЬ УСИЛИВАЕТ...,NaN,"$=61,87р. RTS -2,61% BRENT +0,97%📌 LIVE Карт...",📈 «Ведомости»: «Роснано» могут расформироват...,NaN,NaN,На рынках продолжается фиеста Великобритании...
155,61.600813,62.025917,2022-10-20,0.000200,1,#Еврооблигации #ОФЗ 📝 Выплаты купонов по ОФ...,Акции Tesla упали после публикации отчетности...,🔴🟢 - Снижение в пределах процента на WallStre...,NaN,🔦 Фокус недели #ФН Сегодня\n🇷🇺 Россия: засед...,📈 Росстат сообщил о росте цен промпроизводит...,NaN,NaN,А теперь о рынках. Что интересного там?▪️Инде...
156,61.144888,61.667988,2022-10-21,-0.007401,0,​​ #РейтингиСНГ 🗂 Присвоение / Изменение рей...,NaN,🔴🟢 - Доходности основных суверенных бенчмарко...,NaN,🔦 Фокус недели #ФН Сегодня\n🇷🇺 Россия: засед...,📈 Softline окончательно разделилась на две к...,NaN,NaN,Еще немного о рынках. Всемирно известный эко...


In [133]:
X = df_final.drop(['date_start', 'pnl', 'pnl_sign'], axis=1)
y = df_final['pnl_sign']

In [134]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=121)

In [135]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/buchkovv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [136]:
from scipy.sparse import coo_matrix, hstack
from sklearn.feature_extraction.text import TfidfVectorizer

X_train_transformed = []
X_val_transformed = []
for i in range(len(X_train.columns)):
    tf_idf = TfidfVectorizer(stop_words=russian_stopwords, ngram_range=(1,1))
    
    X_train_transformed.append(coo_matrix(tf_idf.fit_transform(X_train.iloc[:,i].astype(str))))
    X_val_transformed.append(coo_matrix(tf_idf.transform(X_val.iloc[:,i].astype(str))))

In [137]:
for x in X_train_transformed:
    print(x.shape)

(125, 163)
(125, 166)
(125, 20789)
(125, 5791)
(125, 16115)
(125, 2569)
(125, 24279)
(125, 18334)
(125, 2627)
(125, 925)
(125, 41341)


In [138]:
X_train_transformed = hstack(X_train_transformed)
X_val_transformed = hstack(X_val_transformed)
X_train_transformed

<125x133099 sparse matrix of type '<class 'numpy.float64'>'
	with 555498 stored elements in COOrdinate format>

In [139]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(multi_class='auto', solver='lbfgs', verbose=False)
log_reg.fit(X_train_transformed, y_train)

preds = log_reg.predict(X_val_transformed)
accuracy_score(preds, y_val)

0.53125